In [1]:
%cd ../

/Users/hoangle/Projects/Food-Waste-Optimization/experiments_hoangle


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import root_mean_squared_error, r2_score
from torch import Tensor
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold

from utils import Paths

In [3]:
plt.style.use('seaborn-v0_8')
plt.rcParams.update({'font.size': 8})

# Load data and resources

## Load related dim table

In [4]:
dim_lunches = pd.read_excel(Paths.dim_lucnhes(), index_col=None, parse_dates=['date'])
dim_lunches.head()

,date,restaurant,category,meal,pcs
0,2023-01-02,Chemicum,fish,Kalapuikot tillikermaviilikast,78
1,2023-01-02,Chemicum,meat,Uunimakkaraa,165
2,2023-01-02,Chemicum,vegan,Marokkolainen linssipata,84
3,2023-01-03,Chemicum,fish,Herkkulohipihvit,105
4,2023-01-03,Chemicum,fish,Kalapuikot tillikermaviilikast,52


## Load embeddings

In [5]:
map_dish2embd = np.load(Paths().res_dish2embd(), allow_pickle=True).item()
map_cat2embd = np.load(Paths().res_cat2embd(), allow_pickle=True).item()

# Encode features (and target)

In [6]:
cols_X = ['date', 'restaurant', 'category', 'meal']
col_y = 'pcs'

X = dim_lunches[cols_X].copy()
y = dim_lunches[col_y].copy()

## Apply cyclic encoding to 'date'

In [7]:
def get_sin_encoding(x, period: int):
    return np.sin(2 * np.pi * x / period)

def get_cos_encoding(x, period: int):
    return np.cos(2 * np.pi * x / period)

X['weekday_sin'] = get_sin_encoding(X['date'].dt.weekday, 7)
X['weekday_cos'] = get_cos_encoding(X['date'].dt.weekday, 7)

X['day_sin'] = get_sin_encoding(X['date'].dt.day, 30)
X['day_cos'] = get_cos_encoding(X['date'].dt.day, 30)

X['month_sin'] = get_sin_encoding(X['date'].dt.month, 12)
X['month_cos'] = get_cos_encoding(X['date'].dt.month, 12)

X.head()

,date,restaurant,category,meal,weekday_sin,weekday_cos,day_sin,day_cos,month_sin,month_cos
0,2023-01-02,Chemicum,fish,Kalapuikot tillikermaviilikast,0.000000,1.00000,0.406737,0.913545,0.5,0.866025
1,2023-01-02,Chemicum,meat,Uunimakkaraa,0.000000,1.00000,0.406737,0.913545,0.5,0.866025
2,2023-01-02,Chemicum,vegan,Marokkolainen linssipata,0.000000,1.00000,0.406737,0.913545,0.5,0.866025
3,2023-01-03,Chemicum,fish,Herkkulohipihvit,0.781831,0.62349,0.587785,0.809017,0.5,0.866025
4,2023-01-03,Chemicum,fish,Kalapuikot tillikermaviilikast,0.781831,0.62349,0.587785,0.809017,0.5,0.866025


## Encode 'restaurant'

Sep 17: OrdinalEncoding

In [8]:
enc_ord = OrdinalEncoder()

X['restaurant_enc'] = enc_ord.fit_transform(X[['restaurant']]).ravel()

X.head()

,date,restaurant,category,meal,weekday_sin,weekday_cos,day_sin,day_cos,month_sin,month_cos,restaurant_enc
0,2023-01-02,Chemicum,fish,Kalapuikot tillikermaviilikast,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0
1,2023-01-02,Chemicum,meat,Uunimakkaraa,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0
2,2023-01-02,Chemicum,vegan,Marokkolainen linssipata,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0
3,2023-01-03,Chemicum,fish,Herkkulohipihvit,0.781831,0.62349,0.587785,0.809017,0.5,0.866025,0.0
4,2023-01-03,Chemicum,fish,Kalapuikot tillikermaviilikast,0.781831,0.62349,0.587785,0.809017,0.5,0.866025,0.0


## Encode 'category'

Sep 17: OrdinalEncoding

In [9]:
enc_ord = OrdinalEncoder()

X['category_enc'] = enc_ord.fit_transform(X[['category']]).ravel()

X.head()

,date,restaurant,category,meal,weekday_sin,weekday_cos,day_sin,day_cos,month_sin,month_cos,restaurant_enc,category_enc
0,2023-01-02,Chemicum,fish,Kalapuikot tillikermaviilikast,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0,1.0
1,2023-01-02,Chemicum,meat,Uunimakkaraa,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0,2.0
2,2023-01-02,Chemicum,vegan,Marokkolainen linssipata,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0,3.0
3,2023-01-03,Chemicum,fish,Herkkulohipihvit,0.781831,0.62349,0.587785,0.809017,0.5,0.866025,0.0,1.0
4,2023-01-03,Chemicum,fish,Kalapuikot tillikermaviilikast,0.781831,0.62349,0.587785,0.809017,0.5,0.866025,0.0,1.0


## Encode 'meal' 

In [10]:
# Get embedding from meal name
X['meal_enc'] = X['meal'].map(map_dish2embd)

# Concate meal embedding to X
feat_meal = np.vstack(X['meal_enc'].to_list())
cols_feat_meal = [f'meal_name_{i}' for i in range(feat_meal.shape[1])]

df_feat_meal = pd.DataFrame(feat_meal, columns=cols_feat_meal)

X = pd.concat(
    [
        X,
        df_feat_meal
    ],
    axis=1
)

## Keep useful columns

In [11]:
cols = [
    'weekday_sin',
    'weekday_cos',
    'day_sin',
    'day_cos',
    'month_sin',
    'month_cos',
    'restaurant_enc',
    'category_enc',
    *df_feat_meal.columns
]

X = X[cols]

X.head()

,weekday_sin,weekday_cos,day_sin,day_cos,month_sin,month_cos,restaurant_enc,category_enc,meal_name_0,meal_name_1,...,meal_name_90,meal_name_91,meal_name_92,meal_name_93,meal_name_94,meal_name_95,meal_name_96,meal_name_97,meal_name_98,meal_name_99
0,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0,1.0,-41.433788,9.393979,...,-0.464634,-0.760679,-0.415426,-1.402635,-2.390355,2.655900,-0.476949,-0.008931,-0.931314,-2.191599
1,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0,2.0,-40.699654,24.613142,...,2.217119,0.235981,-0.878908,1.233570,2.431796,-0.878301,0.041990,0.555465,2.601398,0.697872
2,0.000000,1.00000,0.406737,0.913545,0.5,0.866025,0.0,3.0,66.496964,-3.386780,...,1.458441,-0.125991,-2.411030,1.717922,0.677590,-0.328726,-0.552619,1.728464,-0.729374,0.004286
3,0.781831,0.62349,0.587785,0.809017,0.5,0.866025,0.0,1.0,-87.672287,31.718786,...,-0.793361,0.096004,-0.306665,-0.674438,-0.584597,1.211040,-0.073525,-0.107445,0.131203,1.393058
4,0.781831,0.62349,0.587785,0.809017,0.5,0.866025,0.0,1.0,-41.433788,9.393979,...,-0.464634,-0.760679,-0.415426,-1.402635,-2.390355,2.655900,-0.476949,-0.008931,-0.931314,-2.191599


# Define NN stuffs

## Define model

In [12]:
class ResNet(nn.Module):
    def __init__(self, n: int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.seq1 = nn.Sequential(
            nn.Linear(n, n),
            nn.LayerNorm(n),
            nn.ReLU(),
        )
        self.seq2 = nn.Sequential(
            nn.LayerNorm(n),
            nn.ReLU(),
        )

    def forward(self, X: Tensor) -> Tensor:
        out = self.seq2(self.seq1(X) + X)

        return out


class Model(nn.Module):
    def __init__(self, d_hid: int = 128, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        # For date
        self.lin1 = nn.Linear(6, d_hid)
        self.block_date = ResNet(d_hid)

        # For restaurant
        self.emb_restaurant = nn.Embedding(3, d_hid)
        self.block_restaurant = ResNet(d_hid)

        # For category
        self.embd_cat = nn.Embedding(5, d_hid)
        self.block_cat = ResNet(d_hid)

        # For meal
        self.lin2 = nn.Linear(100, d_hid)
        self.block_meal = ResNet(d_hid)

        self.block_common = ResNet(d_hid)
        self.lin_common = nn.Linear(d_hid, 1)


    def forward(self, X: Tensor) -> Tensor:
        X_date = X[:, :6]
        X_restaurant = X[:, 6].type(torch.int32)
        X_cat = X[:, 7].type(torch.int32)
        X_meal = X[:, 8:]

        date = self.block_date(self.lin1(X_date))
        restaurant = self.block_restaurant(self.emb_restaurant(X_restaurant))
        category = self.block_cat(self.embd_cat(X_cat))
        meal = self.block_meal(self.lin2(X_meal))

        out = self.block_common(date + restaurant + category + meal)
        out = self.lin_common(out)
        out = nn.functional.relu(out)

        return out
    
# model = Model()

# X_tensor = torch.tensor(X.to_numpy()[:10], dtype=torch.float32)
# out = model(X_tensor)

## Define data and dataloader

In [13]:
class Data(Dataset):
    def __init__(self, X: pd.DataFrame, y: pd.Series, device: str = "cpu") -> None:
        super().__init__()

        self.X_tensor = torch.tensor(X.to_numpy(), device=device, dtype=torch.float32)
        self.y_tensor = torch.tensor(y.to_numpy(), device=device, dtype=torch.float32)

    def __len__(self):
        return len(self.X_tensor)

    def __getitem__(self, indices):
        return self.X_tensor[indices], self.y_tensor[indices]

## Define train flow

In [14]:
EPOCHS = 25
BSZ = 10
LR = 2e-3
D_HID = 64
DEVICE = 'mps'

criterion = nn.MSELoss()


# Start train-val

In [40]:
kf = KFold(n_splits=5, shuffle=True)

scores_rmse, scores_r2 = [], []

for train_idx, val_idx in kf.split(X, y):
    Xtrain, ytrain = X.iloc[train_idx], y.iloc[train_idx]
    Xval, yval = X.iloc[val_idx], y.iloc[val_idx]

    # Start training
    model = Model(D_HID).to(DEVICE)

    loader_train = DataLoader(Data(Xtrain, ytrain, device=DEVICE), batch_size=BSZ, shuffle=True)
    loader_val = DataLoader(Data(Xval, yval, device=DEVICE), batch_size=BSZ, shuffle=False)

    
    optimizer = AdamW(model.parameters(), lr=LR)

    for epch in range(1, EPOCHS + 1):
        model.train()
        for i, (Xtr, ytr) in enumerate(loader_train):
            optimizer.zero_grad()

            out = model(Xtr)

            loss = criterion(out.squeeze(-1), ytr)
            loss.backward()

            optimizer.step()

            # if i % 10 == 0:
            #     print(f"Epch {epch:2d}: train_loss = {loss.item():.4f}")

        if epch % 5 == 0:
            preds, tgts = [], []

            model.eval()
            with torch.no_grad():
                for Xv, yv in loader_val:
                    out = model(Xv)

                    if yv.shape[0] != 1:
                        yv = yv.squeeze(-1)

                    preds.append(out.squeeze(-1))
                    tgts.append(yv)

            pred = torch.concat(preds, dim=-1).cpu()
            tgt = torch.concat(tgts, dim=-1).cpu()

            rmse = root_mean_squared_error(tgt, pred)
            r2 = r2_score(tgt, pred)

            print(f"Epch {epch:2d}: val: rmse: {rmse:.4f} | r2: {r2:.4f}")

    scores_rmse.append(rmse)
    scores_r2.append(r2)

print(f"RMSE: {np.mean(scores_rmse).item():.4f}")
print(f"R2  : {np.mean(scores_r2).item():.4f}")

Epch  5: val: rmse: 57.1650 | r2: 0.5134
Epch 10: val: rmse: 49.7158 | r2: 0.6319
Epch 15: val: rmse: 48.4320 | r2: 0.6507
Epch 20: val: rmse: 48.2440 | r2: 0.6534
Epch 25: val: rmse: 46.6290 | r2: 0.6762
Epch  5: val: rmse: 61.6405 | r2: 0.5030
Epch 10: val: rmse: 54.2364 | r2: 0.6152
Epch 15: val: rmse: 56.1043 | r2: 0.5883
Epch 20: val: rmse: 55.1047 | r2: 0.6028
Epch 25: val: rmse: 56.2999 | r2: 0.5854
Epch  5: val: rmse: 55.8559 | r2: 0.5672
Epch 10: val: rmse: 50.1684 | r2: 0.6508
Epch 15: val: rmse: 51.4179 | r2: 0.6332
Epch 20: val: rmse: 50.4279 | r2: 0.6472
Epch 25: val: rmse: 50.1190 | r2: 0.6515
Epch  5: val: rmse: 60.2370 | r2: 0.4916
Epch 10: val: rmse: 52.4468 | r2: 0.6146
Epch 15: val: rmse: 54.2255 | r2: 0.5880
Epch 20: val: rmse: 53.6209 | r2: 0.5972
Epch 25: val: rmse: 52.6733 | r2: 0.6113
Epch  5: val: rmse: 62.0551 | r2: 0.5321
Epch 10: val: rmse: 54.8644 | r2: 0.6342
Epch 15: val: rmse: 54.5886 | r2: 0.6379
Epch 20: val: rmse: 56.6031 | r2: 0.6107
Epch 25: val: rm